In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.models import load_model
from multiprocessing import cpu_count
from warnings import catch_warnings
from warnings import filterwarnings
from pandas import read_csv
import matplotlib.pyplot as plt
from pandas import DataFrame
import time
#from datetime import datetime
import datetime
import itertools
import mysql.connector
from mysql.connector import connection
from mysql.connector import errorcode
from statistics import mean

In [2]:
station = ["t39","t56","t61","t72","t79","t101","t103"]
# ,"t76","t78","t80"


previous_id = np.zeros(len(station))

In [3]:
def loadData(t):
    myconn = mysql.connector.connect(host = "82.180.142.51",
                                database = "u978805288_land",
                                user = "u978805288_root",
                                password= "ACSL@b123")
    (myconn)

    sql_select_query = "select sensor_id, value from data_log, info_time where data_log.info_id = info_time.info_id AND data_log.triplet_id = '"+t+"' ORDER BY data_log.info_id desc limit 70"
    date_select_query = "select Date_Time from info_time where triplet_id='"+t+"' order by info_id desc limit 1"

    cursor = myconn.cursor()
    cursor.execute(sql_select_query)
    data = cursor.fetchall()

    cursor.execute(date_select_query)
    date = cursor.fetchall()

    myconn.close()
    return data,date[0][0]


In [4]:
triplet_id = "t56"
data, date_value = loadData(triplet_id)
print("Data:")
for row in data:
    print(row)

print("Date:", date_value)
print(len(data))

Data:
('s7', '205')
('s6', '0.00')
('s5', '-0.00,-0.00,-0.00,-0.06,-0.06,-0.06,0.00,0.00,0.00,0')
('s4', '-2.00')
('s3', '0')
('s2', '0.00')
('s1', '0.00,0')
('s7', '180')
('s6', '0.00')
('s5', '-0.00,-0.00,-0.00,-0.06,-0.06,-0.06,0.00,0.00,0.00,0')
('s4', '-2.00')
('s3', '0')
('s2', '0.00')
('s1', '0.00,0')
('s7', '195')
('s6', '0.00')
('s5', '-0.00,-0.00,-0.00,-0.06,-0.06,-0.06,0.00,0.00,0.00,0')
('s4', '-2.00')
('s3', '0')
('s2', '0.00')
('s1', '0.00,0')
('s7', '203')
('s6', '0.00')
('s5', '-0.00,-0.00,-0.00,-0.06,-0.06,-0.06,0.00,0.00,0.00,0')
('s4', '-2.00')
('s3', '0')
('s2', '0.00')
('s1', '0.00,0')
('s7', '118')
('s6', '0.00')
('s5', '-0.00,-0.00,-0.00,-0.06,-0.06,-0.06,0.00,0.00,0.00,0')
('s4', '-2.00')
('s3', '0')
('s2', '0.00')
('s1', '0.00,0')
('s7', '145')
('s6', '0.00')
('s5', '-0.00,-0.00,-0.00,-0.06,-0.06,-0.06,0.00,0.00,0.00,0')
('s4', '-2.00')
('s3', '0')
('s2', '0.00')
('s1', '0.00,0')
('s7', '164')
('s6', '0.00')
('s5', '-0.00,-0.00,-0.00,-0.06,-0.06,-0.06,0.00,0.00

In [5]:
station=["t56"]
# station = ["t39", "t56", "t61", "t72", "t79", "t101", "t103"]

In [6]:
def dataPreprocess(data):
    result = []
    data.reverse()

    combined_data = []
    for i in range(0, len(data), 7):
        row = data[i:i+7]
        sensor_data = []
        for s, d in row:
            # print("R: ",row)
            sid = s
            if sid == 's6':
                continue
            elif sid == 's5':
                x = d.split(',')[:6]
                for j in x:
                    sensor_data.append(float(j))
                x = d.split(',')[-1]
                sensor_data.append(float(x))
            elif sid.startswith('s') and sid != 's5':
                x = d.split(',')[:]
                for j in x:
                    sensor_data.append(float(j))

        combined_data.extend(sensor_data)

    data = np.array(combined_data)
    return combined_data


for id in station:
    data, date_value = loadData(id)
    preprocessed_data = dataPreprocess(data)
    split_rows = [preprocessed_data[i:i+13]
                  for i in range(0, len(preprocessed_data), 13)]
    print("ID:", id, "len: ", len(preprocessed_data))
    for row in split_rows:
        print(row)
# Temp   Humi    Press   Rain    Light   Ax,Ay,Az,Wx,Wy,Wz   Moisture
#API: t56


ID: t56 len:  130
[0.0, 0.0, 0.0, 0.0, -2.0, -0.0, -0.0, -0.0, -0.06, -0.06, -0.06, 0.0, 152.0]
[0.0, 0.0, 0.0, 0.0, -2.0, -0.0, -0.0, -0.0, -0.06, -0.06, -0.06, 0.0, 137.0]
[0.0, 0.0, 0.0, 0.0, -2.0, -0.0, -0.0, -0.0, -0.06, -0.06, -0.06, 0.0, 237.0]
[0.0, 0.0, 0.0, 0.0, -2.0, -0.0, -0.0, -0.0, -0.06, -0.06, -0.06, 0.0, 164.0]
[0.0, 0.0, 0.0, 0.0, -2.0, -0.0, -0.0, -0.0, -0.06, -0.06, -0.06, 0.0, 145.0]
[0.0, 0.0, 0.0, 0.0, -2.0, -0.0, -0.0, -0.0, -0.06, -0.06, -0.06, 0.0, 118.0]
[0.0, 0.0, 0.0, 0.0, -2.0, -0.0, -0.0, -0.0, -0.06, -0.06, -0.06, 0.0, 203.0]
[0.0, 0.0, 0.0, 0.0, -2.0, -0.0, -0.0, -0.0, -0.06, -0.06, -0.06, 0.0, 195.0]
[0.0, 0.0, 0.0, 0.0, -2.0, -0.0, -0.0, -0.0, -0.06, -0.06, -0.06, 0.0, 180.0]
[0.0, 0.0, 0.0, 0.0, -2.0, -0.0, -0.0, -0.0, -0.06, -0.06, -0.06, 0.0, 205.0]


ID: t103 l:  130
[37.2, 26.0, 1042.5, 0.0, 4055.0, -0.96, 0.06, -0.12, 14.7, 10.0, 4.45, 0.0, 232.0]
[37.6, 26.0, 1042.38, 0.0, 4057.5, -0.96, 0.06, -0.12, 12.13, 9.02, 6.59, 0.0, 234.0]
[37.1, 26.0, 1042.14, 0.0, 4147.5, -0.97, 0.05, -0.13, 13.35, 8.23, 6.52, 0.0, 234.0]
[37.8, 25.0, 1042.06, 0.0, 4190.0, -0.97, 0.05, -0.12, 14.88, 8.72, 5.91, 0.0, 237.0]
[38.4, 26.0, 1041.86, 0.0, 4166.67, -0.97, 0.05, -0.13, 14.39, 9.09, 7.62, 0.0, 236.0]
[38.8, 25.0, 1041.68, 0.0, 4360.0, -0.96, 0.05, -0.12, 13.72, 12.99, 5.0, 0.0, 237.0]
[38.6, 25.0, 1041.53, 0.0, 4013.33, -0.96, 0.05, -0.13, 12.99, 9.27, 7.01, 0.0, 233.0]
[38.9, 25.0, 1041.28, 0.0, 4361.67, -0.96, 0.05, -0.13, 12.26, 9.76, 5.98, 0.0, 237.0]
[38.5, 24.0, 1041.21, 0.0, 4351.67, -0.96, 0.06, -0.12, 16.1, 8.54, 5.18, 0.0, 236.0]
[39.2, 23.0, 1041.16, 0.0, 4103.33, -0.97, 0.06, -0.13, 13.72, 9.15, 3.23, 0.0, 236.0]


In [12]:
import tensorflow as tf
data_list = []
count = 0
for i in range(10):
    d = preprocessed_data[count:count+13]
    data_list.append(d)
    count += 13

data_array = np.array(data_list)
data_array = data_array.reshape((-1, 10, 13))
data_tensor = tf.convert_to_tensor(data_array, dtype=tf.float32)

In [ ]:
# Load the first model
STOREDIR = ('/content/drive/MyDrive/Count_data-LMS/urni dhank/Encoder_model/finalmodel/')
model1 = tf.saved_model.load(STOREDIR)
predict_fn = model1.signatures["serving_default"]

# Load the second model
STOR = "/content/drive/MyDrive/Count_data-LMS/urni dhank/"
model2 = tf.keras.models.load_model(STOR+'Encoder_model/website1.h5')

def runModel(data_tensor):
    previous = 1
    try:
        # Run the prediction on the first model
        pred = predict_fn(data_tensor)
        pred = np.argmax(pred)
        pred = np.round(pred).astype(int)

        # Run the prediction on the second model
        dt = data_tensor[:,:,0:5]
        pred1 = model2.predict(dt)

        combined_list = []
        combined_list.extend([round(x,2) for x in pred1.tolist()[0]])
        combined_list.append(pred)

        return (combined_list)
    except Exception as e:
        print(f"Error occurred: {str(e)}")
        return None

In [ ]:
def insertDatabase(prediction, triplet_id):
    connection = mysql.connector.connect(host = "82.180.142.51",
                                database = "u978805288_land",
                                user =   "u978805288_root",
                                password= "ACSL@b123")
    try:
        (connection)
        current_time = date+ datetime.timedelta()
        print('current_time:',current_time)

        val = "(NULL,'" +(triplet_id)+"',"+ str(prediction[0])+","+ str(prediction[1])+","+str(prediction[2])+","+str(prediction[3])+","+str(prediction[4])+","+str(prediction[5])+ ",'"+str(current_time)+ "','no')"

        add_prediction = "INSERT INTO Prediction_Real_Data  VALUES "+val
        print(add_prediction)

        cursor = connection.cursor()
        cursor.execute(add_prediction)
        connection.commit()
        print(cursor.rowcount, "Record inserted successfully into prediction table")
        cursor.close()

    except mysql.connector.Error as error:
        print("Failed to insert record into prediction table {}".format(error))

    finally:
        if connection.is_connected():
            connection.close()

In [ ]:
while True:
  count = 0
  for triplet_id in station:
    data,date = loadData(triplet_id)
    print(date)
    date = date+ datetime.timedelta(minutes=10)
    data = dataPreprocess(data)
    data_list = []
    count = 0
    for i in range(10):
        d = preprocessed_data[count:count+13]
        data_list.append(d)
        count += 13

    data_array = np.array(data_list)
    data_array = data_array.reshape((-1, 10, 13))
    data_tensor = tf.convert_to_tensor(data_array, dtype=tf.float32)
    prediction = runModel(data_tensor)
    print(prediction)
    insertDatabase(prediction,triplet_id)
    print('Enjoy')
  count+=1